<h1><center>Segmenting and Clustering Neighborhoods in Toronto</center></h1>
<h3><center>Federico Sciuca</center></h3>

Building the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1574121600000&hmac=CV78Cmf2C1BDNgZClNp2AYTXMbI6rD7oW2EfZGsxT8Y">

<p>To create the above dataframe:</p>
<ul>
    <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is <strong>Not assigned.</strong></li>
    <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that <strong>M5A</strong> is listed twice and has two neighborhoods: <strong>Harbourfront </strong>and <strong>Regent Park</strong>. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in <strong>row 11 </strong> in the above table.</li>
</ul>

### First of all, I install the libraries I need to scrape the Wikipedia Table

In [1]:
# Get BeautifulSoup Library to scrape the web page and 
!conda install -c anaconda beautifulsoup4 -y

# Get lxlm
!conda install -c anaconda lxml -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
# Install requests library
!conda install -c anaconda requests -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



#### The second step consist in importing the libraries we need to implement the data analysis

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
from bs4 import BeautifulSoup
import requests
import lxml

### Let's start!  
  
##### We can start defining the source we need to scrape and print the Html code

In [4]:
# Define the link for the webpage you need to scrape
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Make the soup
soup = BeautifulSoup(source, 'lxml')

# Print the soup - I commented the code to make the Notebook easy to read
# print(soup.prettify())

From the Html code we need to identify and save in a new variable the code referred to the table with the data we need

In [5]:
# Let's find the table and get the entire tbody
table = soup.find("table", class_="wikitable sortable").tbody

# Print the table in Html code
print(table.prettify())

<tbody>
 <tr>
  <th>
   Postcode
  </th>
  <th>
   Borough
  </th>
  <th>
   Neighbourhood
  </th>
 </tr>
 <tr>
  <td>
   M1A
  </td>
  <td>
   Not assigned
  </td>
  <td>
   Not assigned
  </td>
 </tr>
 <tr>
  <td>
   M2A
  </td>
  <td>
   Not assigned
  </td>
  <td>
   Not assigned
  </td>
 </tr>
 <tr>
  <td>
   M3A
  </td>
  <td>
   <a href="/wiki/North_York" title="North York">
    North York
   </a>
  </td>
  <td>
   <a href="/wiki/Parkwoods" title="Parkwoods">
    Parkwoods
   </a>
  </td>
 </tr>
 <tr>
  <td>
   M4A
  </td>
  <td>
   <a href="/wiki/North_York" title="North York">
    North York
   </a>
  </td>
  <td>
   <a href="/wiki/Victoria_Village" title="Victoria Village">
    Victoria Village
   </a>
  </td>
 </tr>
 <tr>
  <td>
   M5A
  </td>
  <td>
   <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
    Downtown Toronto
   </a>
  </td>
  <td>
   <a href="/wiki/Regent_Park" title="Regent Park">
    Harbourfront
   </a>
  </td>
 </tr>
 <tr>
  <td>
   M6A
  </td>
  

We can note that every single row is defined between the tr tag. Let's find the rows

In [6]:
# Define the rows
rows = table.find_all("tr")

The columns names are definded between the tag th

In [7]:
# Identify the columns name
columns = [v.text.replace("\n", "") for v in rows[0].find_all("th")]
print(columns)

['Postcode', 'Borough', 'Neighbourhood']


Now it's time to create a new Pandas DataFrame and assign to the columns the columns name we have just extract

In [8]:
# Define the DataFrame
df = pd.DataFrame(columns=columns)

# Check if the DataFrame has been created correctly
df

,Postcode,Borough,Neighbourhood


The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.  
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [9]:
for i in range(1, len(rows)):
    # Identify the cells for each row
    tds = rows[i].find_all("td")
    
    # If the "Borough" is missing, go to the next row, otherwise assign to the variable values the value of each cell
    if tds[1].text != "Not assigned":
        values = [tds[0].text.replace("\n", ""), 
                  tds[1].text.replace("\n", ""), 
                  tds[2].text.replace("\n", "")]
    else:
        continue
    
    # Append the new values to the DataFrame
    df = df.append(pd.Series(values, index=columns), ignore_index=True)
    df.head(15)

In [10]:
df.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Check how many rows and columns we have:

In [11]:
df.shape

(210, 3)

Let's check how many unique values each column has

In [12]:
df.T.apply(lambda x: x.nunique(), axis=1)

Postcode         103
Borough           11
Neighbourhood    208
dtype: int64

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.  
  
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [13]:
# define a for loop that replace the "Not assigned" values in the "Neighbourhood" column with the value of the "Borough" from the same row
for i in range(0, len(df)):
    
    if df.iloc[i]["Neighbourhood"] == 'Not assigned':
        df.iloc[i]["Neighbourhood"] = df.iloc[i]["Borough"]
    else:
        pass

In [14]:
df.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Now we need to group the DataFrame by the "Postcode" and the "Borough" aggregating the "Neighbourhood" as a comma separated format

In [15]:
df_grouped = df.groupby(["Postcode", "Borough"])["Neighbourhood"].agg([('Neighbourhood', ', '.join)]).reset_index()
df_grouped

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Let's check how many rows and columns we have

In [16]:
df_grouped.shape

(103, 3)